In [89]:
import pandas as pd
import numpy as np

In [90]:
ls data

XBTUSDC_1.csv     XBTUSDC_15.csv    XBTUSDC_60.csv
XBTUSDC_1440.csv  XBTUSDC_5.csv     XBTUSDC_720.csv


In [91]:
df = pd.read_csv('data/XBTUSDC_60.csv', header=None,
                 names=['time', 'open', 'high', 'low', 'close', 'volume', 'trades'])
df.set_index('time', inplace=True)
df.index = pd.DatetimeIndex(df.index * 1e+9)

In [97]:
df.head()

,open,high,low,close,volume,trades
time,,,,,,
2020-01-08 15:00:00,8300.00,8300.00,8300.00,8300.00,0.002410,2
2020-01-08 16:00:00,8217.51,8217.51,8217.51,8217.51,0.012290,2
2020-01-08 17:00:00,7900.01,7949.99,7870.00,7949.99,0.052487,10
2020-01-08 18:00:00,7949.99,8000.00,7870.99,7870.99,0.038592,8
2020-01-08 20:00:00,7870.98,7999.99,7870.00,7999.99,0.044715,5


In [118]:
class Trader1:
    # Bollinger: https://www.investopedia.com/trading/using-bollinger-bands-to-gauge-trends/#:~:text=Using%20the%20bands%20as%20overbought,have%20deviated%20from%20the%20mean.
    def __init__(self, n_cycles, n_devs, qty):
        # EMA = Closing price x multiplier + EMA (previous day) x (1-multiplier)
        # 
        # BOLU=MA(TP,n)+m∗σ[TP,n]
        # BOLD=MA(TP,n)−m∗σ[TP,n]
        # where:
        # BOLU=Upper Bollinger Band
        # BOLD=Lower Bollinger Band
        # MA=Moving average
        # TP (typical price)=(High+Low+Close)÷3
        # n=Number of days in smoothing period
        # m=Number of standard deviations
        # σ[TP,n]=Standard Deviation over last n periods of TP
        self.N = n_cycles
        self.M = n_devs
        self.qty = qty
        self.prices = []
        
        self.sma = None
        self.upper = None
        self.lower = None
        self.std = None
    
        self.position = 0
        self.trades = []
    
    def update(self, cycle, ohlc: dict):
        _, high, low, close = self._unpack(ohlc)
        
        price = (high + low + close) / 3.
        
        self.prices.append(price)
        if len(self.prices) > self.N:
            self.prices.pop(0)
            
        if len(self.prices) == self.N:
            self.sma = np.mean(self.prices)
            self.std = np.std(self.prices)
            self.upper = self.sma + self.std
            self.lower = self.sma - self.std
    
    def trade(self) -> float:  # return number of shares to trade
        if len(self.prices) < self.N:
            return None
    
        # buy
        if self.prices[-1] < self.lower and self.position <= 0:
            self.position += self.qty
            return self.qty
    
        # sell
        if self.prices[-1] > self.upper and self.position >= 0:
            self.position -= self.qty
            return -self.qty
        
    
    @staticmethod
    def _unpack(ohlc: dict):
        return ohlc['open'], ohlc['high'], ohlc['low'], ohlc['close']

In [56]:
assert all((df.high - df.low) >= 0)

In [102]:
df.close.rolling(3).std().mean(), df.close.rolling(3).std().std()

(114.68306082114233, 172.63341179012235)

In [134]:
def trade(df, model):
    trades = pd.DataFrame(index=df.index)
    trades['qty'] = [0] * len(trades)
    
    for i, (cycle, ohlc) in enumerate(df.iterrows()):
        model.update(cycle, ohlc)
        qty = model.trade()
        if qty:
            trades.iloc[i] = qty
    return trades

def account(df, trades):
    qty_left = trades.qty.sum()
    last_price = df.iloc[-1].close
    
    pnl = qty_left * last_price + np.dot(trades.qty.values[:-1], df.close.values[1:])
    return pnl

In [126]:
trades = trade(df, Trader1(20, 1, 0.001))

In [131]:
trades[trades.qty != 0].size

755

In [135]:
account(df, trades)

-94.45592999999977